In [ ]:
### Dit is oude, niet-werkende code die geen onderdeel vormt van de pipeline.
### This is old code that is not-working and not a part of the pipeline.

In [1]:
import pandas as pd
from transformers import AutoTokenizer, pipeline
import transformers
import torch
from tqdm.auto import tqdm
import time
import os
import json
import sys
import gc
from tqdm import tqdm

In [2]:
def read_dict_from_json(json_file_path):
    """
    Reads a dictionary from a JSON file.

    Parameters:
    json_file_path (str): Path to the JSON file.

    Returns:
    dict: Dictionary read from the JSON file, or None if an error occurred.
    """

    # Check if the file exists
    if not os.path.exists(json_file_path):
        print(f"Error: The file {json_file_path} does not exist.")
        return None

    try:
        # Open and read the JSON file
        with open(json_file_path, 'r') as file:
            data = json.load(file)
            return data

    except json.JSONDecodeError:
        print(f"Error: The file {json_file_path} does not contain valid JSON.")
  
    except Exception as e:
        print(f"An error occurred: {e}")

    return None

In [ ]:
# id_to_axiom_sentence = read_dict_from_json('/data4T/jieying/data/ontologies/GeoFault/merge_geofault_xml_del_pt.owl_id_to_axiom_sentence.json')
# id_to_axiom_sentence = read_dict_from_json('/data4T/jieying/data/ontologies/disease/ontology-2023-09-05_16-12-34.owl_branch.clinical_finding.disease.owl_id_to_axiom_sentence.json')

# file_path = '/data4T/jieying/data/ontologies/GeoFault/GeoFaultBenchmark.pkl_finalAnalysedResult.pkl'
# file_path = '/data4T/jieying/data/ontologies/foodon/benchmark/achive/benchmark_raw_grouped_manual_part_1.csv_withoutFAISS_finalAnalysedResult.pkl'

In [3]:
# Get command line arguments
model_7b = "meta-llama/Llama-2-7b-chat-hf"

if len(sys.argv) < 4:
    raise ValueError("Insufficient arguments provided. Please provide the model name, JSON file path, data file path, and GPU ID.")

# model_7b = sys.argv[1]
json_file_path = sys.argv[1]
file_path = sys.argv[2]
gpu_id = sys.argv[3]

# Read data
id_to_axiom_sentence = read_dict_from_json(json_file_path)

# Keeping only the first two rows
df = pd.read_pickle(file_path)

ValueError: Insufficient arguments provided. Please provide the model name, JSON file path, data file path, and GPU ID.

In [ ]:
# Function to generate the prompt for each row
# def generate_prompt_for_row(query, candidate_ids, id_to_sentence_map):
#     # Start the prompt with the query from the DataFrame
#     prompt = "### Input:"
#     prompt += "Could you please find the most relevant setences from the following candidate sentences with respect to the reference sentence. Please provide the ranking in the format: \"The ranking of candidates is: [ranked list of IDs].Please answer briefly using candidate IDs, separated by commas.\"\n\n"
#     prompt += f"Here is the reference sentence:\n\"{query}\"\n\n"
#     prompt += "Candidates:\n\n"

#     for id in candidate_ids:
#         sentence = id_to_sentence_map.get(id, "Sentence not found for ID {}".format(id))
#         new_line = f"ID: {id} - \"{sentence}\"\n"
#         if len(prompt) + len(new_line) < 2024:  # Check to keep under token limit
#             prompt += new_line
#         else:
#             break  # Stop adding more candidates if limit is reached

#     prompt += "### Response:"
#     return prompt

In [ ]:
def generate_prompt_for_row(query, candidate_ids, id_to_sentence_map, k):
    # Start the prompt with the query from the DataFrame
    prompt = "### Input:"
    prompt += "Could you please find the most relevant setences from the following candidate sentences with respect to the reference sentence. Please provide the ranking in the format: \"The ranking of candidates is: [ranked list of IDs].Please answer briefly using candidate IDs, separated by commas.\"\n\n"
    prompt += f"Here is the reference sentence:\n\"{query}\"\n\n"
    prompt += "Candidates:\n\n"

    # Include up to 'k' candidate IDs in the prompt
    for id in candidate_ids[:k]:  # Only iterate over the first 'k' IDs
        sentence = id_to_sentence_map.get(id, "Sentence not found for ID {}".format(id))
        prompt += f"ID: {id} - \"{sentence}\"\n"

    prompt += "### Response:"
    return prompt

In [ ]:
# Apply the function to each row in the DataFrame
# df['SBERT Prompt'] = df.apply(lambda row: generate_prompt_for_row(row['Query'], row['SBERT_Ranking'].keys(), id_to_axiom_sentence), axis=1)
# df['BERT Prompt'] = df.apply(lambda row: generate_prompt_for_row(row['Query'], row['BERT_Ranking'].keys(), id_to_axiom_sentence), axis=1)

In [14]:
# Check if GPU is available and specify GPU ID
device = f"cuda:{gpu_id}" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model_7b = "meta-llama/Meta-Llama-3-8B"

tokenizer_7b = AutoTokenizer.from_pretrained(model_7b)

# Specify the device explicitly in the pipeline
pipeline_7b = transformers.pipeline(
    "text-classification",
    model=model_7b,
    torch_dtype=torch.float16,
    device=device
)

Using device: cpu


OSError: meta-llama/Meta-Llama-3-8B is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [ ]:
# Function to generate response and measure computation time
def generate_response(query):
    start_time = time.time()

    llama_prompt = "[INST]" + query + "[/INST]"
    with torch.no_grad():
        sequences = pipeline_7b(
            llama_prompt,
            do_sample=True,
            top_k=10,
            num_return_sequences=1,
            eos_token_id=tokenizer_7b.eos_token_id,
            max_length=9000,
        )
    
    end_time = time.time()
    computation_time = end_time - start_time

    # print(sequences[0]['generated_text'])
    return sequences[0]['generated_text'], computation_time

In [ ]:
# Process and record results
final_output_file = file_path + "_llama7b.pkl"

In [ ]:
# def process_queries_in_batches(listA, batch_size=10):
#     results = []
#     for batch_start in tqdm(range(0, len(listA), batch_size), desc="Processing batches"):
#         batch_end = min(batch_start + batch_size, len(listA))
#         queries = listA[batch_start:batch_end]

#         # Process each query and record the time
#         batch_results = []
#         for query in queries:
#             start_time = time.time()
#             response = generate_response(query)
#             end_time = time.time()
#             time_taken = end_time - start_time
#             batch_results.append((response, time_taken))

#         results.extend(batch_results)

In [ ]:
# # Generate prompts
# ks = [10, 20, 50]

# for k in ks:
#     df[f'SBERT Prompt {k}'] = df.apply(
#         lambda row: generate_prompt_for_row(
#             row['Annotation Text'] if 'Annotation Text' in df.columns else row['Query'],
#             list(row['SBERT_Ranking'].keys()),
#             id_to_axiom_sentence, k
#         ),
#         axis=1
#     )

#     df[f'BERT Prompt {k}'] = df.apply(
#         lambda row: generate_prompt_for_row(
#             row['Annotation Text'] if 'Annotation Text' in df.columns else row['Query'],
#             list(row['BERT_Ranking'].keys()),
#             id_to_axiom_sentence, k
#         ),
#         axis=1
#     )

#     df[f'SapBERT Prompt {k}'] = df.apply(
#         lambda row: generate_prompt_for_row(
#             row['Annotation Text'] if 'Annotation Text' in df.columns else row['Query'],
#             list(row['SapBERT_Ranking'].keys()),
#             id_to_axiom_sentence, k
#         ),
#         axis=1
#     )

# df['SBERT Prompt'] = df.apply(lambda row: generate_prompt_for_row(row['Query'], row['SBERT_Ranking'].keys(), id_to_axiom_sentence), axis=1)
# df['BERT Prompt'] = df.apply(lambda row: generate_prompt_for_row(row['Query'], row['BERT_Ranking'].keys(), id_to_axiom_sentence), axis=1)
# df['SapBERT Prompt'] = df.apply(lambda row: generate_prompt_for_row(row['Query'], row['SapBERT_Ranking'].keys(), id_to_axiom_sentence), axis=1)

In [ ]:
def process_queries_in_chunks(df, query_column, answer_column, final_output_file, chunk_size=100, batch_size=10, save_every=20):
    for chunk_start in range(0, len(df), chunk_size):
        chunk_end = min(chunk_start + chunk_size, len(df))
        chunk = df.iloc[chunk_start:chunk_end]

        for batch_start in range(0, len(chunk), batch_size):
            batch_end = min(batch_start + batch_size, len(chunk))
            queries = chunk[query_column][batch_start:batch_end].tolist()

            batch_results = []

            for i, query in enumerate(queries):
                if chunk.iloc[i][answer_column] == 'tba':
                    response = generate_response(query)
                else:
                    response = chunk.iloc[i][answer_column]
                batch_results.append(response)

            for i, result in enumerate(batch_results):
                if(chunk_start + batch_start + 1<len(df)):
                    df[answer_column].iloc[chunk_start + batch_start + i] = result

            if (chunk_start + batch_start) % save_every == 0:
                df.to_pickle(final_output_file)

    df.to_pickle(final_output_file)

    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    gc.collect()

In [ ]:
# process_queries_in_chunks(df,'BERT Prompt','Answer_BERT_llama_7b',final_output_file)
# process_queries_in_chunks(df,'SBERT Prompt','Answer_SBERT_llama_7b',final_output_file)
# process_queries_in_chunks(df,'SapBERT Prompt','Answer_SapBERT_llama_7b',final_output_file)

ks = [10, 20]

for k in ks:
    # Generate prompts for each k
    df[f'SBERT Prompt {k}'] = df.apply(
        lambda row: generate_prompt_for_row(
            row['Annotation Text'] if 'Annotation Text' in df.columns else row['Query'],
            list(row['SBERT_Ranking'].keys()),
            id_to_axiom_sentence, k
        ),
        axis=1
    )

    df[f'BERT Prompt {k}'] = df.apply(
        lambda row: generate_prompt_for_row(
            row['Annotation Text'] if 'Annotation Text' in df.columns else row['Query'],
            list(row['BERT_Ranking'].keys()),
            id_to_axiom_sentence, k
        ),
        axis=1
    )

    df[f'SapBERT Prompt {k}'] = df.apply(
        lambda row: generate_prompt_for_row(
            row['Annotation Text'] if 'Annotation Text' in df.columns else row['Query'],
            list(row['SapBERT_Ranking'].keys()),
            id_to_axiom_sentence, k
        ),
        axis=1
    )

    # Check if each column exists, and if not, create it with default value 'tba'
    if f'Answer_BERT_llama_7b_{k}' not in df.columns:
        df[f'Answer_BERT_llama_7b_{k}'] = 'tba'

    if f'Answer_SBERT_llama_7b_{k}' not in df.columns:
        df[f'Answer_SBERT_llama_7b_{k}'] = 'tba'

    if f'Answer_SapBERT_llama_7b_{k}' not in df.columns:
        df[f'Answer_SapBERT_llama_7b_{k}'] = 'tba'

    # Process the generated prompts
    process_queries_in_chunks(df, f'BERT Prompt {k}', f'Answer_BERT_llama_7b_{k}', final_output_file)
    df.drop(f'BERT Prompt {k}', axis=1, inplace=True)
    process_queries_in_chunks(df, f'SBERT Prompt {k}', f'Answer_SBERT_llama_7b_{k}', final_output_file)
    df.drop(f'SBERT Prompt {k}', axis=1, inplace=True)
    process_queries_in_chunks(df, f'SapBERT Prompt {k}', f'Answer_SapBERT_llama_7b_{k}', final_output_file)s
    df.drop(f'SapBERT Prompt {k}', axis=1, inplace=True)

In [ ]:
# batched_results = process_queries_in_batches(df['BERT Prompt'])
# df['Answer_BERT_llama_7b'], df['Time_BERT_prompt_llama2_7b'] = zip(*batched_results)

# sbert_batched_results = process_queries_in_batches(df['SBERT Prompt'])
# df['Answer_SBERT_llama_7b'], df['Time_SBERT_prompt_llama2_7b'] = zip(*sbert_batched_results)

# sapbert_batched_results = process_queries_in_batches(df['SapBERT Prompt'])
# df['Answer_SapBERT_llama_7b'], df['Time_SapBERT_prompt_llama2_7b'] = zip(*sapbert_batched_results)

df.to_pickle(final_output_file)

# Display the first few rows of the DataFrame to verify
df

In [ ]:
######################################################
# import pandas as pd
# from transformers import AutoTokenizer, pipeline
# import transformers
# import torch
# from tqdm.auto import tqdm
# import time
# import os
# import json

In [ ]:
# def read_dict_from_json(json_file_path):
#     """
#     Reads a dictionary from a JSON file.

#     Parameters:
#     json_file_path (str): Path to the JSON file.

#     Returns:
#     dict: Dictionary read from the JSON file, or None if an error occurred.
#     """

#     # Check if the file exists
#     if not os.path.exists(json_file_path):
#         print(f"Error: The file {json_file_path} does not exist.")
#         return None

#     try:
#         # Open and read the JSON file
#         with open(json_file_path, 'r') as file:
#             data = json.load(file)
#             return data

#     except json.JSONDecodeError:
#         print(f"Error: The file {json_file_path} does not contain valid JSON.")
#     except Exception as e:
#         print(f"An error occurred: {e}")

#     return None

In [ ]:
# model_7b = "meta-llama/Llama-2-7b-chat-hf"
# # id_to_axiom_sentence = read_dict_from_json('/data4T/jieying/data/ontologies/GeoFault/merge_geofault_xml_del_pt.owl_id_to_axiom_sentence.json')
# id_to_axiom_sentence = read_dict_from_json('/data4T/jieying/data/ontologies/foodon/foodon-merged.owl_id_to_axiom_sentence.json')

# # file_path = '/data4T/jieying/data/ontologies/GeoFault/GeoFaultBenchmark.pkl_finalAnalysedResult.pkl'
# file_path = '/data4T/jieying/data/ontologies/foodon/benchmark/achive/benchmark_raw_grouped_manual_part_1.csv_withoutFAISS_finalAnalysedResult.pkl'

# # Keeping only the first two rows
# df = pd.read_pickle(file_path)

# # Update the pipeline creation
# device = 0 if torch.cuda.is_available() else -1  # GPU if available, otherwise CPU
# model = AutoModelForCausalLM.from_pretrained(model_7b).to(device)
# tokenizer_7b = AutoTokenizer.from_pretrained(model_7b)
# pipeline_7b = TextGenerationPipeline(model=model, tokenizer=tokenizer_7b, device=device)

In [ ]:
# # Function to generate the prompt for each row
# def generate_prompt_for_row(query, candidate_ids, id_to_sentence_map):
#     # Start the prompt with the query from the DataFrame
#     prompt = "### Input:"
#     prompt += "Could you please find the most relevant setences from the following candidate sentences with respect to the reference sentence. Please provide the ranking in the format: \"The ranking of candidates is: [ranked list of IDs].Please answer briefly using candidate IDs, separated by commas.\"\n\n"
#     prompt += f"Here is the reference sentence:\n\"{query}\"\n\n"
#     prompt += "Candidates:\n\n"

#     for id in candidate_ids:
#         sentence = id_to_sentence_map.get(id, "Sentence not found for ID {}".format(id))
#         new_line = f"ID: {id} - \"{sentence}\"\n"
#         if len(prompt) + len(new_line) < 2024:  # Check to keep under token limit
#             prompt += new_line
#         else:
#             break  # Stop adding more candidates if limit is reached

#     prompt += "### Response:"
#     return prompt

In [ ]:
# # Apply the function to each row in the DataFrame
# # df['SBERT Prompt'] = df.apply(lambda row: generate_prompt_for_row(row['Query'], row['SBERT_Ranking'].keys(), id_to_axiom_sentence), axis=1)
# # df['BERT Prompt'] = df.apply(lambda row: generate_prompt_for_row(row['Query'], row['BERT_Ranking'].keys(), id_to_axiom_sentence), axis=1)

In [ ]:
# # Function to generate response and measure computation time
# def generate_response(query):
#     start_time = time.time()
#     llama_prompt = "[INST]" + query + "[/INST]"
#     sequences = pipeline_7b(
#         llama_prompt,
#         do_sample=True,
#         top_k=10,
#         num_return_sequences=1,
#         eos_token_id=tokenizer_7b.eos_token_id,
#         max_length=4000,
#     )
#     end_time = time.time()
#     computation_time = end_time - start_time
#     # print(sequences[0]['generated_text'])
#     return sequences[0]['generated_text'], computation_time

In [ ]:
# # # Batch processing of queries
# # def process_queries_in_batches(df, batch_size=30):
# #     results = []
# #     for batch_start in tqdm(range(0, len(df), batch_size), desc="Processing batches"):
# #         batch_end = min(batch_start + batch_size, len(df))
# #         queries = df['SBERT Prompt'][batch_start:batch_end]
# #         batch_results = [generate_response(query) for query in queries]
# #         results.extend(batch_results)
# #     return results

# # Batch processing of queries
# def process_queries_in_batches(listA, batch_size=10):
#     results = []
#     for batch_start in tqdm(range(0, len(listA), batch_size), desc="Processing batches"):
#         batch_end = min(batch_start + batch_size, len(df))
#         queries = listA[batch_start:batch_end]
#         batch_results = [generate_response(query) for query in queries]
#         results.extend(batch_results)
#     return results

In [ ]:
# df['SBERT Prompt'] = df.apply(lambda row: generate_prompt_for_row(row['Annotation Text'], row['SBERT_Ranking'].keys(), id_to_axiom_sentence), axis=1)
# df['BERT Prompt'] = df.apply(lambda row: generate_prompt_for_row(row['Annotation Text'], row['BERT_Ranking'].keys(), id_to_axiom_sentence), axis=1)

# # Apply the function to each row in the DataFrame in batches and record results
# batched_results = process_queries_in_batches(df['BERT Prompt'])
# df['Answer_BERT_llama_7b'], df['Time_BERT_prompt_llama2_7b'] = zip(*batched_results)

# sbert_batched_results = process_queries_in_batches(df['SBERT Prompt'])
# df['Answer_SBERT_llama_7b'], df['Time_SBERT_prompt_llama2_7b'] = zip(*batched_results)

# # Save the DataFrame
# final_output_file = file_path+"_llama.pkl"
# df.to_pickle(final_output_file)

# # Display the first few rows of the DataFrame to verify
# df